In [3]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [2]:
#!pip install -U sentence-transformers

  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=08c13faab392c09479b41da80e916639831423f1b974f04116d2c02d995c2673
  Stored in directory: c:\users\tayfu\appdata\local\pip\cache\wheels\0c\b6\fb\2289a932c365293ad865fc1fe9d2db694d5584241c6d670874
Successfully built sentence-transformers
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0
    Uninstalling torch-1.9.0:
      Successfully uninstalled torch-1.9.0


In [7]:
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm
import transformers

In [4]:
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.', 
    'The quick brown fox jumps over the lazy dog.']
sentence_embeddings = model.encode(sentences)

In [5]:
DATA_FOLDER = 'data/rel_ext_data/'

In [8]:
df_train = pd.read_csv(DATA_FOLDER +'train.csv', encoding='utf-8')
df_train.head()

,text,intent
0,tell me the expiration date for my current cre...,expiration_date
1,would you disconnect from my phone,sync_device
2,could you please track my package,order_status
3,any travel alerts for canada,travel_alert
4,i want to report fraudulent activity on my ame...,report_fraud


In [9]:
df_val = pd.read_csv(DATA_FOLDER +'val.csv', encoding='utf-8')
df_val.head()

,text,intent
0,hey,greeting
1,put laundry on my chore list,todo_list_update
2,go into whisper mode,whisper_mode
3,when do i need to change my motor oil again,oil_change_when
4,what is the insurance plan i am enrolled in,insurance


In [10]:
df_test = pd.read_csv(DATA_FOLDER +'test.csv', encoding='utf-8')
df_test.head()

,text,intent
0,thanks so much ai,thank_you
1,i will be traveling to lima alert my bank,travel_notification
2,say again please,repeat
3,what is needed to cook lasagna,ingredients_list
4,give me the pressure for the tires on my car,tire_pressure


In [11]:
train_embeds = model.encode(df_train['text'])

In [12]:
valid_embeds = model.encode(df_val['text'])

In [13]:
test_embeds = model.encode(df_test['text'])

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


def classify_it(classifier):
    clf = classifier.fit(train_embeds, df_train['intent'])
    print("{} Train-Acc:{:.3f} Test-Acc:{:.3f} Validation-Acc:{:.3f}".format(classifier.__class__.__name__, 
                          clf.score(train_embeds, df_train['intent']),
                          clf.score(test_embeds, df_test['intent']),
                          clf.score(valid_embeds, df_val['intent'])
                         ));

In [24]:
classify_it(LogisticRegression())

LogisticRegression Train-Acc:0.979 Test-Acc:0.936 Validation-Acc:0.937


In [25]:
classify_it(RandomForestClassifier())

RandomForestClassifier Train-Acc:1.000 Test-Acc:0.875 Validation-Acc:0.879


In [30]:
for c in [0.01, 0.1, 1, 10, 100]:
    print(c, end=' ')
    classify_it(LogisticRegression(C=c))

0.01 LogisticRegression Train-Acc:0.913 Test-Acc:0.880 Validation-Acc:0.874
0.1 LogisticRegression Train-Acc:0.928 Test-Acc:0.894 Validation-Acc:0.891
1 LogisticRegression Train-Acc:0.979 Test-Acc:0.936 Validation-Acc:0.937
10 LogisticRegression Train-Acc:0.999 Test-Acc:0.949 Validation-Acc:0.947
100 LogisticRegression Train-Acc:1.000 Test-Acc:0.950 Validation-Acc:0.949


C:\Users\tayfu\anaconda3\envs\nlu\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [34]:
for c in [200]:
    print(c, end=' ')
    classify_it(LogisticRegression(C=c, max_iter=200))

200 LogisticRegression Train-Acc:1.000 Test-Acc:0.950 Validation-Acc:0.950


In [35]:
for c in [1000]:
    print(c, end=' ')
    classify_it(LogisticRegression(C=c, max_iter=200))

1000 LogisticRegression Train-Acc:1.000 Test-Acc:0.949 Validation-Acc:0.950
